# Hazardous Chemicals in Cosmetics

In [23]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import warnings
warnings.filterwarnings("ignore")

In [24]:
# Read in the data set
input_data = "Resources/cosmetics_data.csv"
cosmetics_df = pd.read_csv(input_data, low_memory=False)
cosmetics_df.head(1)

,CDPHId,ProductName,CSFId,CSF,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,...,CasNumber,ChemicalId,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount
0,2,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,NaN,NaN,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),53,...,13463-67-7,6,Titanium dioxide,06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1


# Data Cleaning

In [25]:
# Remove unnecessary columns
columns_to_drop = ["CDPHId", "CSFId", "CompanyId", "PrimaryCategoryId", "SubCategoryId", "CasId", "ChemicalId"]
cosmetics_cleaned_df = cosmetics_df.drop(columns=columns_to_drop)
cosmetics_cleaned_df.head(1)

,ProductName,CSF,CompanyName,BrandName,PrimaryCategory,SubCategory,CasNumber,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount
0,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,NaN,New Avon LLC,AVON,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",13463-67-7,Titanium dioxide,06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1


In [26]:
# Change dates to datetime objects
cosmetics_cleaned_df["InitialDateReported"] = pd.to_datetime(cosmetics_cleaned_df["InitialDateReported"])
cosmetics_cleaned_df["MostRecentDateReported"] = pd.to_datetime(cosmetics_cleaned_df["MostRecentDateReported"])
cosmetics_cleaned_df["DiscontinuedDate"] = pd.to_datetime(cosmetics_cleaned_df["DiscontinuedDate"])
cosmetics_cleaned_df["ChemicalCreatedAt"] = pd.to_datetime(cosmetics_cleaned_df["ChemicalCreatedAt"])
cosmetics_cleaned_df["ChemicalUpdatedAt"] = pd.to_datetime(cosmetics_cleaned_df["ChemicalUpdatedAt"])
cosmetics_cleaned_df["ChemicalDateRemoved"] = pd.to_datetime(cosmetics_cleaned_df["ChemicalDateRemoved"])
cosmetics_cleaned_df.head(1)

,ProductName,CSF,CompanyName,BrandName,PrimaryCategory,SubCategory,CasNumber,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount
0,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,NaN,New Avon LLC,AVON,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",13463-67-7,Titanium dioxide,2009-06-17,2013-08-28,2011-02-01,2009-07-09,2009-07-09,NaT,1


In [27]:
# Check shape
cosmetics_cleaned_df.shape

(114635, 15)

In [28]:
# Check for duplicates
print(cosmetics_cleaned_df.duplicated().sum())

7701


In [29]:
# Drop duplicates and check shape again
cosmetics_cleaned_df = cosmetics_cleaned_df.drop_duplicates()
cosmetics_cleaned_df.shape

(106934, 15)

In [34]:
# Filter the dataset where the 'CasId' is missing (NaN)
missing_cas_df = cosmetics_cleaned_df[cosmetics_cleaned_df['CasNumber'].isna()]

# Select relevant columns like 'ChemicalName' and 'CasId' to inspect
missing_cas_df = missing_cas_df[['ChemicalName', 'CasNumber']]

# Select only the unique chemical names with missing CAS numbers
unique_missing_cas_df = missing_cas_df[['ChemicalName']].drop_duplicates()

# Group by 'ChemicalName' and count occurrences
missing_cas_count_df = missing_cas_df.groupby('ChemicalName').size().reset_index(name='Count')

# Merge the two DataFrames on 'ChemicalName'
missing_cas_chemicals_df = pd.merge(unique_missing_cas_df, missing_cas_count_df, on='ChemicalName', how='inner')

# Sort by the count column (assuming the count column is named 'Count')
missing_cas_chemicals_df = missing_cas_chemicals_df.sort_values(by='Count', ascending=False)
missing_cas_chemicals_df

,ChemicalName,Count
4,"Silica, crystalline (airborne particles of res...",2461
2,Retinyl esters,1751
10,"Mineral oils, untreated and mildly treated",510
11,Cocamide DEA,234
5,Titanium dioxide,208
22,Cosmetic talc,186
16,"Aloe vera, whole leaf extract",141
30,"Titanium dioxide (airborne, unbound particles ...",84
3,Coffee,77
34,Isopropyl alcohol manufacture using strong acids,61


In [33]:
# Standardize name for all retinyl esters
cosmetics_cleaned_df.loc[cosmetics_cleaned_df["ChemicalName"] == "Retinyl palmitate", "ChemicalName"] = "Retinyl esters"
cosmetics_cleaned_df.loc[cosmetics_cleaned_df["ChemicalName"] == "Retinol/retinyl esters, when in daily dosages in excess of 10,000 IU, or 3,000 retinol equivalents.", "ChemicalName"] = "Retinyl esters"
cosmetics_cleaned_df.loc[cosmetics_cleaned_df["ChemicalName"] == "Retinyl acetate", "ChemicalName"] = "Retinyl esters"

In [37]:
# Standardize name for titanium dioxide
cosmetics_cleaned_df.loc[cosmetics_cleaned_df["ChemicalName"] == "Titanium dioxide (airborne, unbound particles of respirable size)", "ChemicalName"] = "Titanium dioxide"

In [38]:
# Standardize name for talc
cosmetics_cleaned_df.loc[cosmetics_cleaned_df["ChemicalName"] == "Cosmetic talc", "ChemicalName"] = "Talc"
cosmetics_cleaned_df.loc[cosmetics_cleaned_df["ChemicalName"] == "Talc (powder)", "ChemicalName"] = "Talc"
cosmetics_cleaned_df.loc[cosmetics_cleaned_df["ChemicalName"] == "Talc containing asbestiform fibers", "ChemicalName"] = "Talc"

In [39]:
# Standardize name for cocamide DEA
cosmetics_cleaned_df.loc[cosmetics_cleaned_df["ChemicalName"] == "Cocamide diethanolamine", "ChemicalName"] = "Cocamide DEA"